## Configuración de ambiente de trabajo

```bash
pip install --upgrade pip
```

```bash
pip install pyjanitor matplotlib==3.5.1 missingno numpy pandas pyreadr seaborn session-info upsetplot==0.6.1
```

or 

```bash
pip install -r requirements.txt
```

## Importar librerías

In [161]:
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot
from fs import open_fs
from pathlib import Path
import shutil # Necesario para la operación de movimiento de archivos

## Importar funciones personalizadas

## Configurar el aspecto general de las gráficas del proyecto

In [162]:
%matplotlib inline

sns.set(
    rc={
        "figure.figsize": (10, 10)
    }
)

sns.set_style("whitegrid")

## Operar con valores faltantes

### Python

In [163]:
# print(None or True,
#       None or True,
#       None == None,
#       None is None,
#       type(None),
#       sep='\n')
      

### NumPy

In [164]:
# print(
#     np.nan  or True,
#     np.nan is np.nan,
#     np.nan == np.nan,
#     np.nan / 2,
#     type(np.nan),
#     np.isnan(np.nan),
#     sep='\n'    
# )

### Pandas

In [165]:
# test_missing_df = pd.DataFrame.from_dict(
#     data=dict(
#         x=[0, 1, np.nan, np.nan, None],
#         y=[0, 1, pd.NA, np.nan, None]
#     )
# )
# print(test_missing_df)

In [166]:
# test_missing_df.isna()

In [167]:
# test_missing_df.isnull()

In [168]:
# test_missing_df.x.isnull()

In [169]:
# pd.Series([1, np.nan])

In [170]:
# pd.Series([pd.to_datetime('2022-01-01'), np.nan])

In [171]:
# pd.Series([-1]).isnull()


## Cargar los conjuntos de datos

In [172]:
# 1️⃣ Crear la carpeta .kaggle en tu home (usuario local)
import os
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

# 2️⃣ Descargar el dataset
!kaggle datasets download -d kumargh/pimaindiansdiabetescsv

# 3️⃣ Descomprimir
!yes | unzip -o pimaindiansdiabetescsv.zip -d pima_diabetes



Dataset URL: https://www.kaggle.com/datasets/kumargh/pimaindiansdiabetescsv
License(s): CC0-1.0
pimaindiansdiabetescsv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pimaindiansdiabetescsv.zip
  inflating: pima_diabetes/pima-indians-diabetes.csv  
yes: standard output: Broken pipe


In [173]:
# --- PARTE 1: UBICAR Y MOVER EL ARCHIVO ---

# 1. Obtener la ruta del directorio donde se está ejecutando este Notebook.
# Esto te dará algo como: Path('/home/paco/datos_faltantes/curso-datos-faltantes-main/jupyter')
current_notebook_dir = Path.cwd()
print(f"Directorio actual del Notebook: {current_notebook_dir}")

# 2. Construir la ruta al archivo de ORIGEN (donde está el CSV después de descomprimir).
# Es 'pima_diabetes/pima-indians-diabetes.csv' RELATIVO al directorio del Notebook.
source_file_path = current_notebook_dir / 'pima_diabetes' / 'pima-indians-diabetes.csv'
print(f"Ruta de origen esperada: {source_file_path}")

# 3. Construir la ruta al directorio de DESTINO ('data/').
# El directorio 'data' está UN NIVEL ARRIBA del directorio del Notebook (jupyter/)
# y luego se entra en 'data/'.
# current_notebook_dir.parent te lleva a 'curso-datos-faltantes-main/'.
# Luego, le añades 'data'.
destination_dir = current_notebook_dir.parent / 'data'
print(f"Directorio de destino esperado: {destination_dir}")

# 4. Construir la ruta COMPLETA del archivo en su destino.
destination_file_path = destination_dir / 'pima-indians-diabetes.csv'
print(f"Ruta de destino final: {destination_file_path}")

# 5. Mover el archivo SÓLO SI EXISTE en el origen.
if source_file_path.exists():
    # shutil.move es como el comando 'mv' de Linux o 'move' de Windows.
    # Mueve el archivo de source_file_path a destination_file_path.
    shutil.move(source_file_path, destination_file_path)
    print(f"\n¡Éxito! Archivo '{source_file_path.name}' movido a '{destination_file_path}'.")
else:
    print(f"\nError: El archivo de origen '{source_file_path}' no fue encontrado.")
    print("Asegúrate de que el archivo 'pimaindiansdiabetescsv.zip' se haya descargado y descomprimido correctamente en 'jupyter/pima_diabetes/'.")

# --- PARTE 2: CARGAR EL ARCHIVO CON PANDAS DESDE LA NUEVA UBICACIÓN ---

# Ahora que el archivo está en 'data/', la ruta para Pandas debe reflejar eso.
# Desde 'jupyter/', para llegar a 'data/', subimos un nivel ('../') y entramos en 'data/'.
final_csv_load_path_for_pandas = '../data/pima-indians-diabetes.csv'

try:
    diabetes_df = pd.read_csv(
    final_csv_load_path_for_pandas,
    sep=",",
    names=[
        "pregnancies",
        "glucose",
        "blood_pressure",
        "skin_thickness",
        "insulin",
        "bmi",
        "diabetes_pedigree_function",
        "age",
        "outcome",
    ]
    )
    print(diabetes_df.head())
    
except FileNotFoundError:
    print(f"\nError: No se pudo cargar el archivo CSV desde '{final_csv_load_path_for_pandas}'.")
    print("Verifica que el archivo haya sido movido correctamente y que la ruta sea correcta.")

Directorio actual del Notebook: /home/paco/datos_faltantes/curso-datos-faltantes-main/jupyter
Ruta de origen esperada: /home/paco/datos_faltantes/curso-datos-faltantes-main/jupyter/pima_diabetes/pima-indians-diabetes.csv
Directorio de destino esperado: /home/paco/datos_faltantes/curso-datos-faltantes-main/data
Ruta de destino final: /home/paco/datos_faltantes/curso-datos-faltantes-main/data/pima-indians-diabetes.csv

¡Éxito! Archivo 'pima-indians-diabetes.csv' movido a '/home/paco/datos_faltantes/curso-datos-faltantes-main/data/pima-indians-diabetes.csv'.
   pregnancies  glucose  blood_pressure  skin_thickness  insulin   bmi  \
0            6      148              72              35        0  33.6   
1            1       85              66              29        0  26.6   
2            8      183              64               0        0  23.3   
3            1       89              66              23       94  28.1   
4            0      137              40              35      168  43

### Pima Indians Diabetes

### naniar (oceanbuoys, pedestrian, riskfactors)

#### Crear unidades de información de los conjuntos de datos

In [174]:
# --- Configuración (ajusta si es necesario) ---
# Obtener el directorio padre de tu notebook (curso-datos-faltantes-main/)
# Esto es esencial para ubicar la carpeta 'data' de forma robusta.
project_root_dir = Path.cwd().parent
data_dir = project_root_dir / 'data' # La ruta completa a tu directorio 'data'

# Nombres de los datasets (sin extensión .rda)
# Asegúrate de que estos nombres coincidan con los nombres de los objetos R dentro de los archivos .rda
datasets_names = ["oceanbuoys", "pedestrian", "riskfactors"]
extension = ".rda"

# Diccionario para almacenar los DataFrames
datasets_dfs = {}

# --- Lectura y Conversión ---

print(f"Buscando archivos .rda en: {data_dir}\n")

for dataset_name in datasets_names:
    # Construir la ruta completa del archivo .rda
    dataset_file_path = data_dir / f"{dataset_name}{extension}"

    if dataset_file_path.exists():
        print(f"Leyendo '{dataset_file_path.name}'...")
        try:
            # Leer el archivo .rda
            # .get(dataset_name) intenta extraer un objeto llamado 'dataset_name' del archivo .rda
            df = pyreadr.read_r(str(dataset_file_path)).get(dataset_name)
            datasets_dfs[f"{dataset_name}_df"] = df
            print(f"  -> '{dataset_name}_df' creado con éxito.")
        except Exception as e:
            print(f"  Error al leer '{dataset_file_path.name}' con pyreadr: {e}")
            print(f"  Asegúrate de que el nombre del objeto R dentro del archivo .rda sea '{dataset_name}'.")
            print("  Si el nombre del objeto R es diferente al nombre del archivo, necesitarás ajustar .get().")
    else:
        print(f"Advertencia: Archivo '{dataset_file_path.name}' no encontrado en '{data_dir}'. Saltando.")

# --- Verificación ---
print("\n--- DataFrames cargados ---")
if datasets_dfs:
    for df_name, df in datasets_dfs.items():
        print(f"\nDataFrame: {df_name}")
        print(df.head())
        print(f"Forma: {df.shape}")
else:
    print("No se cargaron DataFrames.")


Buscando archivos .rda en: /home/paco/datos_faltantes/curso-datos-faltantes-main/data

Leyendo 'oceanbuoys.rda'...
  -> 'oceanbuoys_df' creado con éxito.
Leyendo 'pedestrian.rda'...
  -> 'pedestrian_df' creado con éxito.
Leyendo 'riskfactors.rda'...
  -> 'riskfactors_df' creado con éxito.

--- DataFrames cargados ---

DataFrame: oceanbuoys_df
     year  latitude  longitude  sea_temp_c  air_temp_c   humidity  wind_ew  \
0  1997.0       0.0     -110.0   27.590000       27.15  79.599998     -6.4   
1  1997.0       0.0     -110.0   27.549999       27.02  75.800003     -5.3   
2  1997.0       0.0     -110.0   27.570000       27.00  76.500000     -5.1   
3  1997.0       0.0     -110.0   27.620001       26.93  76.199997     -4.9   
4  1997.0       0.0     -110.0   27.650000       26.84  76.400002     -3.5   

   wind_ns  
0      5.4  
1      5.3  
2      4.5  
3      2.5  
4      4.1  
Forma: (736, 8)

DataFrame: pedestrian_df
  hourly_counts           date_time  year    month  month_day week

In [175]:
print(datasets_dfs)

{'oceanbuoys_df':        year  latitude  longitude  sea_temp_c  air_temp_c   humidity  wind_ew  \
0    1997.0       0.0     -110.0   27.590000   27.150000  79.599998     -6.4   
1    1997.0       0.0     -110.0   27.549999   27.020000  75.800003     -5.3   
2    1997.0       0.0     -110.0   27.570000   27.000000  76.500000     -5.1   
3    1997.0       0.0     -110.0   27.620001   26.930000  76.199997     -4.9   
4    1997.0       0.0     -110.0   27.650000   26.840000  76.400002     -3.5   
..      ...       ...        ...         ...         ...        ...      ...   
731  1993.0      -2.0     -110.0   24.780001   24.620001  90.800003     -4.4   
732  1993.0      -2.0     -110.0   24.879999   24.500000  91.099998     -3.0   
733  1993.0      -2.0     -110.0   25.120001   24.889999  89.699997     -4.0   
734  1993.0      -2.0     -110.0   25.160000   24.930000  89.500000     -4.3   
735  1993.0      -2.0     -110.0   25.139999   24.770000  87.900002     -4.6   

     wind_ns  
0     

#### Incluir conjuntos de datos en nuestro ambiente local

In [176]:
# --- Mover DataFrames del Diccionario al Ámbito Global y Limpiar ---

print("\n--- Convirtiendo DataFrames de diccionario a variables globales ---")
if datasets_dfs:
    # Esta línea es la clave: extrae cada par clave-valor del diccionario
    # y los establece como variables independientes en tu entorno de Jupyter.
    locals().update(**datasets_dfs)
    print("DataFrames movidos al ámbito global (ej. 'oceanbuoys_df').")

    # Una vez que los DataFrames están como variables globales,
    # el diccionario datasets_dfs ya no es necesario y se puede eliminar para liberar memoria.
    del datasets_dfs
    print("Diccionario 'datasets_dfs' eliminado.")
else:
    print("No hay DataFrames cargados para mover.")

# --- Verificación de Variables Globales ---

print("\n--- Verificación de acceso a DataFrames como variables globales ---")
# Ahora puedes acceder a los DataFrames directamente por sus nombres
# Por ejemplo:
try:
    if 'oceanbuoys_df' in locals():
        print("Variable 'oceanbuys_df' existe.")
        print(oceanbuoys_df.head())
        print(f"Forma de 'oceanbuoys_df': {oceanbuoys_df.shape}")
    else:
        print("La variable 'oceanbuoys_df' no se pudo crear (verifica nombres de archivos/objetos R).")

    if 'pedestrian_df' in locals():
        print("Variable 'pedestrian_df' existe.")
        print(pedestrian_df.head())
        print(f"Forma de 'pedestrian_df': {pedestrian_df.shape}")

    if 'riskfactors_df' in locals():
        print("Variable 'riskfactors_df' existe.")
        print(riskfactors_df.head())
        print(f"Forma de 'riskfactors_df': {riskfactors_df.shape}")

except NameError as e:
    print(f"\nError al intentar acceder a una variable global: {e}")
    print("Esto puede ocurrir si el nombre del objeto R dentro del archivo .rda no coincide con el 'dataset_name' utilizado en la lista.")
except Exception as e:
    print(f"\nOcurrió un error inesperado durante la verificación: {e}")


--- Convirtiendo DataFrames de diccionario a variables globales ---
DataFrames movidos al ámbito global (ej. 'oceanbuoys_df').
Diccionario 'datasets_dfs' eliminado.

--- Verificación de acceso a DataFrames como variables globales ---
Variable 'oceanbuys_df' existe.
     year  latitude  longitude  sea_temp_c  air_temp_c   humidity  wind_ew  \
0  1997.0       0.0     -110.0   27.590000       27.15  79.599998     -6.4   
1  1997.0       0.0     -110.0   27.549999       27.02  75.800003     -5.3   
2  1997.0       0.0     -110.0   27.570000       27.00  76.500000     -5.1   
3  1997.0       0.0     -110.0   27.620001       26.93  76.199997     -4.9   
4  1997.0       0.0     -110.0   27.650000       26.84  76.400002     -3.5   

   wind_ns  
0      5.4  
1      5.3  
2      4.5  
3      2.5  
4      4.1  
Forma de 'oceanbuoys_df': (736, 8)
Variable 'pedestrian_df' existe.
  hourly_counts           date_time  year    month  month_day week_day  hour  \
0           883 2016-01-01 00:00:00  2

### Verificar carga

In [177]:
oceanbuoys_df.shape, oceanbuoys_df.shape, oceanbuoys_df.shape, oceanbuoys_df.shape, oceanbuoys_df.shape

((736, 8), (736, 8), (736, 8), (736, 8), (736, 8))

#### Descargar y cargar los conjuntos de datos

## Tabulación de valores faltantes

### Resúmenes básicos de valores faltantes

#### Número total de valores completos (sin observaciones faltantes)

#### Número total de valores faltantes

### Resúmenes tabulares de valores faltantes

#### Variables / Columnas

##### Resumen por variable

###### Tabulación del resumen por variable

#### Casos / Observaciones / Filas

##### Resúmenes por caso

###### Tabulación del resumen por caso

### Intervalos de valores faltantes

### _Run length_ de valores faltantes

## Visualización inicial de valores faltantes

### Variable

### Casos / Observaciones / Filas

## Codificación de valores faltantes

<div class="alert alert-warning", role="alert">
    <b style="font-size: 1.5em;">🚧 Advertencia</b>
    <p>
    Al igual que cada persona es una nueva puerta a un mundo diferente, los <b>valores faltantes</b> existen en diferentes formas y colores. Al trabajar con valores faltantes será crítico entender sus distintas representaciones. A pesar de que el conjunto de datos de trabajo pareciera que no contiene valores faltantes, deberás ser capaz de ir más allá de lo observado a simple vista para remover el manto tras el cual se esconde lo desconocido.
    </p>
</div>

### Valores comúnmente asociados a valores faltantes

#### Cadenas de texto

In [118]:
common_na_strings = (
    "missing",
    "NA",
    "N A",
    "N/A",
    "#N/A",
    "NA ",
    " NA",
    "N /A",
    "N / A",
    " N / A",
    "N / A ",
    "na",
    "n a",
    "n/a",
    "na ",
    " na",
    "n /a",
    "n / a",
    " a / a",
    "n / a ",
    "NULL",
    "null",
    "",
    "?",
    "*",
    ".",
)

#### Números

In [119]:
common_na_numbers = (-9, -99, -999, -9999, 9999, 66, 77, 88, -1)

### ¿Cómo encontrar los valores comúnmente asociados a valores faltantes?

In [120]:
missing_data_example_df = pd.DataFrame.from_dict(
    dict(
        x = [1, 3, "NA", -99, -98, -99],
        y = ["A", "N/A", "NA", "E", "F", "G"],
        z = [-100, -99, -98, -101, -1, -1]
    )
)

missing_data_example_df

,x,y,z
0,1,A,-100
1,3,N/A,-99
2,NA,NA,-98
3,-99,E,-101
4,-98,F,-1
5,-99,G,-1


#### Revisar tipos de datos

#### Revisar valores únicos de los datos

### Sustituyendo valores comúnmente asociados a valores faltantes

#### Sustitución desde la lectura de datos

#### Sustitución global

#### Sustitución dirigida

## Conversión de valores faltantes implícitos a explícitos

<div class="alert alert-warning", role="alert">
    <b style="font-size: 1.5em;">🚧 Advertencia</b>
    <br>
    <br>
    <p>
        <i>
        "<b>Implícito</b> se refiere a todo aquello que se entiende que está incluido
        pero sin ser expresado de forma directa o explícitamente."
        </i>
    </p>
    <p>
    Un <code>valor faltante implícito</code> indica que el valor faltante <b>debería estar incluido</b>
    en el conjunto de datos del análisis, <b>sin que éste lo diga</b> o lo <b>especifique</b>.
    Por lo general, son valores que podemos encontrar al pivotar nuestros datos
    o contabilizar el número de apariciones de combinaciones de las variables de estudio.
    </p>
</div>

In [121]:
implicit_to_explicit_df = pd.DataFrame.from_dict(
    data={
        "name": ["lynn", "lynn", "lynn", "zelda"],
        "time": ["morning", "afternoon", "night", "morning"],
        "value": [350, 310, np.nan, 320]
    }
)

implicit_to_explicit_df

,name,time,value
0,lynn,morning,350.0
1,lynn,afternoon,310.0
2,lynn,night,NaN
3,zelda,morning,320.0


### Estrategias para la identificación de valores faltantes implícitos

#### Pivotar la tabla de datos

#### Cuantificar ocurrencias de n-tuplas

### Exponer filas faltantes implícitas a explícitas

<div class="alert alert-info">
    <b style="font-size: 1.5em;">📘 Información</b>
    <p>
       <a href="https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.complete.complete", class="alert-link"><code>janitor.complete()</code></a> está modelada a partir de la función <a href="https://tidyr.tidyverse.org/reference/complete.html", class="alert-link"><code>complete()</code></a> del paquete <a href="https://tidyr.tidyverse.org/index.html", class="alert-link"><code>tidyr</code></a> y es un <i>wrapper</i> alrededor de <a href="https://pyjanitordevs.github.io/pyjanitor/api/functions/#janitor.functions.expand_grid.expand_grid", class="alert-link"><code>janitor.expand_grid()</code></a>, <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html", class="alert-link"><code>pd.merge()</code></a> y <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html", class="alert-link"><code>pd.fillna()</code></a>. En cierto modo, es lo contrario de <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html", class="alert-link"><code>pd.dropna()</code></a>, ya que expone implícitamente las filas que faltan.
    </p>
    <p>
    Son posibles combinaciones de nombres de columnas o una lista/tupla de nombres de columnas, o incluso un  diccionario de nombres de columna y nuevos valores.
    </p>
    <p>
    Las columnas <a href="https://pandas.pydata.org/docs/user_guide/advanced.html"><code>MultiIndex</code></a> no son complatibles.
    </p>
</div>

#### Exponer n-tuplas de valores faltantes

Ejemplo, encontrar los pares faltantes de `name` y `time`.

#### Limitar la exposición de n-tuplas de valores faltantes

#### Rellenar los valores faltantes

#### Limitar el rellenado de valores faltantes implícitos

## Tipos de valores faltantes

### _Missing Completely At Random_ (MCAR)

### _Missing At Random_ (MAR)

### _Missing Not At Random_ (MNAR)

## Concepto y aplicación de la matriz de sombras (_i.e._, _shadow matrix_)

 ### Construcción de la matriz de sombras

### Utilizar función de utilería `bind_shadow_matrix()`

### Explorar estadísticos utilizando las nuevas columnas de la matriz de sombras

## Visualización de valores faltantes en una variable

## Visualización de valores faltantes en dos variables

## Correlación de nulidad

## Eliminación de valores faltantes

<div class="alert alert-warning", role="alert">
    <b style="font-size: 1.5em;">🚧 Advertencia</b>
    <p>
    La eliminación de valores faltantes <b>asume</b> que los valores faltantes están perdidos
    completamente al azar (<code>MCAR</code>). En cualquier otro caso, realizar una
    eliminación de valores faltantes podrá ocasionar <b>sesgos</b> en los
    análisis y modelos subsecuentes.
    </p>
</div>

Primero observa el número total de observaciones y variables que tiene tu conjunto de datos.

### _Pairwise deletion_ (eliminación por pares)

### _Listwise Deletion or Complete Case_ (Eliminación por lista o caso completo)

#### Con base en 1 columna

#### Con base en 2 o más columnas

### Representación gráfica tras la eliminación de los valores faltantes

## Imputación básica de valores faltantes

### Imputación con base en el contexto

In [122]:
implicit_to_explicit_df = pd.DataFrame(
    data={
        "name": ["lynn", np.nan, "zelda", np.nan, "shadowsong", np.nan],
        "time": ["morning", "afternoon", "morning", "afternoon", "morning", "afternoon",],
        "value": [350, 310, 320, 350, 310, 320]
    }
)

implicit_to_explicit_df

,name,time,value
0,lynn,morning,350
1,NaN,afternoon,310
2,zelda,morning,320
3,NaN,afternoon,350
4,shadowsong,morning,310
5,NaN,afternoon,320


### Imputación de un único valor

## Continúa aprendiendo sobre el manejo de valores faltantes

<div class="alert alert-success">
    <b style="font-size: 1.5em;">✅ ¡Felicidades por terminar el curso!</b>
    <p>
Has aprendido bastante sobre la exploración y manipulación de valores faltantes.
    </p>
    <p>
Empezaste conociento las principales operaciones al trabajar con valores faltantes. Ahora, eres consciente de que estas operaciones no son universales y cada software decide tratar a los valores faltantes a su conveniencia.
    </p>
    <p>
Y, hablando de conveniencias, comenzaste tu camino en la exploración de valores faltantes a través de una representación universal de qué es lo que faltaba. No obstante, no pasó mucho para darte cuenta de que los valores faltantes pueden existir en formas muy variables. Incluso, en formas en las que no sabemos que nos faltan estos valores en sí mismos. 
    </p>
    <p>
Con los valores faltantes ya expuestos, te conviertes en una persona capaz de explorarlos en profundidad de forma estadística y visual. Entendiendo así, los distintos mecanismos que pueden tener los valores faltantes: MCAR, MAR y MNAR.
    </p>
    <p>
A su vez, aprendiste las bases sobre cómo tratarlos a través de la eliminación de elementos o la imputación de valores de una forma básica y sencilla. Por lo tanto, necesitarás continuar tu camino de aprendizaje con un curso que te permita profundizar en estas técnicas de tratamiento para valores faltantes.
    </p>
    <p>
Te recomiendo continuar con mi <a href="https://platzi.com/cursos/datos-faltantes-imputacion/">Curso de Manejo de Datos Faltantes: Imputación</a>. Estoy seguro de que tus habilidades adquiridas hasta el momento mejorarán, permitiéndote realizar análisis cada vez más complejos y cercanos al mundo real.
    </p>
    <p>
    Con mucha alegría por tu logro,
   Jesús Vélez Santiago
    </p>
    
</div>

## Información de sesión

In [123]:
session_info.show()

KeyError: 'backports'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=694a3d08-7f18-421d-9e2f-c2820a79680e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>